# End-to-End Handwritten Text Recognition

This notebook demonstrates the complete OCR pipeline that converts handwritten document images into digital text using a CNN–BiLSTM–CTC model with optional language model decoding.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import joblib
import cv2
from PIL import Image
import tempfile
import gradio as gr
from pyctcdecode import build_ctcdecoder
import pyctcdecode.decoder as decoder_module
import pyctcdecode.language_model as language_model_module
import kenlm


In [ ]:
MODEL_DIR = "models/htr_model"
KENLM_BINARY = "models/iam_lm_5gram.binary"

model = keras.models.load_model(os.path.join(MODEL_DIR, "htr_model.keras"), compile=False)
vocab_list = joblib.load(os.path.join(MODEL_DIR, "vocab_list.pkl"))


In [ ]:
decoder_module.kenlm = kenlm
language_model_module.kenlm = kenlm

decoder = build_ctcdecoder(
    labels=vocab_list,
    kenlm_model_path=KENLM_BINARY
)


In [ ]:
# ============================================
# Line segmentation (your projection method)
# ============================================
def segment_lines_projection(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    _, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((2, 50), np.uint8)
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    hist = np.sum(closed, axis=1)
    threshold = np.max(hist) * 0.1

    lines = []
    in_line = False
    start = 0
    min_line_height = 10
    for y, val in enumerate(hist):
        if val > threshold and not in_line:
            in_line = True
            start = y
        elif val <= threshold and in_line:
            in_line = False
            end = y
            if end - start >= min_line_height:
                lines.append((start, end))

    line_imgs = []
    for i, (y1, y2) in enumerate(lines):
        y1 = max(y1 - 10, 0)
        y2 = min(y2 + 10, img.shape[0])
        cropped = img[y1:y2, :]
        pil_img = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        line_imgs.append(pil_img)

    return line_imgs

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 64, 800

def preprocess_image(img):
    img = img.convert("L")
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img = np.array(img, dtype=np.float32) / 255.0
    img = np.expand_dims(img, axis=(0, -1))
    return img


In [ ]:
def decode_greedy(pred):
    pred = np.squeeze(pred)
    best_path = np.argmax(pred, axis=-1)
    return "".join([vocab_list[i] for i in best_path if i < len(vocab_list)])

def decode_with_kenlm(pred):
    pred = np.squeeze(pred)
    return decoder.decode(pred)


In [ ]:
# --- inject kenlm into pyctcdecode modules (required) ---
decoder_module.kenlm = kenlm
language_model_module.kenlm = kenlm
print("✅ KenLM successfully linked to pyctcdecode")

# ============================================
# Paths — update these to your actual Drive paths if needed
# ============================================
MODEL_DIR = "/content/drive/MyDrive/htr_final_model_20251110_110833"   # update if different
KENLM_ARPA = "/content/drive/MyDrive/iam_lm_5gram.arpa"                 # your 5-gram ARPA

# ============================================
# Load your HTR model + vocab
# ============================================
print("🔄 Loading model and vocab...")
model = keras.models.load_model(os.path.join(MODEL_DIR, "htr_model.keras"), compile=False)
vocab_list = joblib.load(os.path.join(MODEL_DIR, "vocab_list.pkl"))
print(f"✅ Model loaded! Vocab size: {len(vocab_list)}")

# ============================================
# Build KenLM decoder (5-gram)
# ============================================
print("🔄 Building 5-gram KenLM decoder...")
decoder = build_ctcdecoder(labels=vocab_list, kenlm_model_path=KENLM_ARPA)
print("✅ 5-gram KenLM decoder ready!")

# ============================================
# Preprocessing — keep same size as model expects
# ============================================
IMG_HEIGHT, IMG_WIDTH = 64, 800   # IMPORTANT: width=800, height=64 matches model

def preprocess_image(img: Image.Image):
    # img is PIL.Image
    img = img.convert("L")
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))   # (800, 64)
    arr = np.array(img, dtype=np.float32) / 255.0
    arr = np.expand_dims(arr, axis=-1)          # (64,800,1)
    arr = np.expand_dims(arr, axis=0)           # (1,64,800,1)
    return arr

# ============================================
# Decoding helpers
# ============================================
def decode_greedy(pred):
    pred = np.squeeze(pred)                     # (timesteps, vocab_size)
    best_path = np.argmax(pred, axis=-1)
    # collapse repeats & remove blank if using blank index — but your original code did simple mapping
    # We'll do basic collapsing of repeats and remove indices >= len(vocab)
    chars = []
    prev = None
    for idx in best_path:
        if idx == prev:
            continue
        prev = idx
        if idx < len(vocab_list):
            chars.append(vocab_list[idx])
    return ''.join(chars)

def decode_with_kenlm(pred):
    pred = np.squeeze(pred)
    # pyctcdecode expects logits or probabilities per timestep; we pass pred as-is
    return decoder.decode(pred)

# ============================================
# Gradio prediction function (matches your original UI)
# ============================================
def recognize_text(uploaded_image, ground_truth):
    if uploaded_image is None:
        return [], "", "", "⚠️ Please upload a handwriting image."

    # Save temporary image file so projection uses OpenCV
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp:
        uploaded_image.save(tmp.name)
        image_path = tmp.name

    # Segment lines
    line_images = segment_lines_projection(image_path)
    if not line_images:
        return [], "", "", "❌ No lines detected. Try a clearer image."

    results = []
    greedy_lines = []
    lm_lines = []

    for line_img in line_images:
        img_input = preprocess_image(line_img)          # shape (1,64,800,1)
        preds = model.predict(img_input, verbose=0)     # model output shape e.g. (1, T, V)

        greedy_text = decode_greedy(preds)
        lm_text = decode_with_kenlm(preds)

        # show both on gallery
        results.append((line_img, f"🧾 Greedy: {greedy_text}\n📖 LM: {lm_text}"))
        greedy_lines.append(greedy_text)
        lm_lines.append(lm_text)

    greedy_final = "\n".join(greedy_lines)
    lm_final = "\n".join(lm_lines)

    if isinstance(ground_truth, str) and ground_truth.strip():
        return (
            results,
            greedy_final,
            lm_final,
            f"WER (Greedy): {wer(ground_truth, greedy_final):.2%} | CER: {cer(ground_truth, greedy_final):.2%}\n"
            f"WER (LM): {wer(ground_truth, lm_final):.2%} | CER: {cer(ground_truth, lm_final):.2%}"
        )
    else:
        return results, greedy_final, lm_final, "ℹ️ No ground truth provided."

# ============================================
# Gradio UI (matches your earlier design)
# ============================================
with gr.Blocks(title="HTR + 5-Gram KenLM Handwriting Recognition") as interface:
    gr.Markdown("## 📝 Handwritten Text Recognition (Greedy vs 5-gram KenLM)")
    gr.Markdown("Upload a handwritten image. The system will segment lines and show both raw (CTC greedy) and LM-enhanced predictions.")

    upload = gr.Image(type="pil", label="📤 Upload Handwritten Image")
    ground_truth_input = gr.Textbox(label="✅ Ground Truth (optional for accuracy)", lines=4)
    recognize_btn = gr.Button("🧠 Recognize Text")

    gallery = gr.Gallery(label="📸 Line Predictions", columns=1, preview=True)
    greedy_output = gr.Textbox(label="🔹 Greedy Decode (No LM)")
    lm_output = gr.Textbox(label="🔹 5-gram KenLM Decode (With LM)")
    accuracy_output = gr.Textbox(label="📊 Accuracy Comparison")

    recognize_btn.click(
        fn=recognize_text,
        inputs=[upload, ground_truth_input],
        outputs=[gallery, greedy_output, lm_output, accuracy_output]
    )

interface.launch(share=True)
